## Save and serialize models with Keras

In [41]:
import tensorflow as tf

tf.keras.backend.clear_session()

### Saving Sequential models or Functional models

In [42]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784, ), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, name='predictions')(x)

In [43]:
model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')
model.summary()

Model: "3_layer_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [44]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255
dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset_train = dataset_train.batch(64).shuffle(60000,
                                                reshuffle_each_iteration=True)

dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
dataset_test = dataset_test.batch(64)

In [45]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=['acc'])
history = model.fit(dataset_train, epochs=5)

Train for 938 steps
Epoch 1/5
938/938 [==============================] - 2s 2ms/step - loss: 0.3285 - acc: 0.9062
Epoch 2/5
938/938 [==============================] - 2s 2ms/step - loss: 0.1504 - acc: 0.9561
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 0.1085 - acc: 0.9674
Epoch 4/5
938/938 [==============================] - 2s 2ms/step - loss: 0.0876 - acc: 0.9742
Epoch 5/5
938/938 [==============================] - 1s 2ms/step - loss: 0.0741 - acc: 0.9776


In [46]:
predictions = model.predict(dataset_test)

### Whole-model saving

In [47]:
model.save('model_save.h5')

In [48]:
new_model = keras.models.load_model('model_save.h5')

In [49]:
import numpy as np

new_predictions = model.predict(dataset_test)

In [50]:
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

### Export to SavedModel

In [51]:
model.save('model_save_tf', save_format='tf')

INFO:tensorflow:Assets written to: model_save_tf/assets


In [52]:
new_model = keras.models.load_model('model_save_tf')

In [53]:
new_predictions = new_model.predict(dataset_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

### Architecture-only saving

In [54]:
config = model.get_config()
reinitialized_model = keras.Model.from_config(config)
reinitialized_model.summary()

Model: "3_layer_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [55]:
new_predictions = reinitialized_model.predict(dataset_test)
assert abs(np.sum(predictions - new_predictions)) > 0.

In [56]:
json_config = model.to_json()
reinitialized_model = keras.models.model_from_json(json_config)

In [57]:
new_predictions = reinitialized_model.predict(dataset_test)
assert abs(np.sum(predictions - new_predictions)) > 0.

### Weights-only saving

In [58]:
weights = model.get_weights()
model.set_weights(weights)

In [59]:
config = model.get_config()
weights = model.get_weights()

new_model = keras.Model.from_config(config)
new_model.set_weights(weights)

new_predictions = new_model.predict(dataset_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

<p> Como criamos o modelo de config e weights, não é possivel treinar o modelo, precisamos compila novamente </p>

In [60]:
new_model.fit(dataset_train, epochs=5)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [61]:
json_config = model.to_json()
with open('model_config_t1.json', 'w') as json_file:
    json_file.write(json_config)

model.save_weights('weights_t1.h5')

In [62]:
with open('model_config_t1.json') as json_file:
    json_config = json_file.read()
new_model = keras.models.model_from_json(json_config)
new_model.load_weights('weights_t1.h5')

In [63]:
new_predictions = new_model.predict(dataset_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

### Recomendado

In [64]:
model.save('model_save.h5')
del model
model = keras.models.load_model('model_save.h5')

In [65]:
new_predictions = model.predict(dataset_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

### Saving and Loading of Subclassed Models

In [66]:
class ThreeLayerMLP(keras.Model):
    def __init__(self, name=None):
        super(ThreeLayerMLP, self).__init__(name=name)
        self.dense_1 = layers.Dense(64, activation='relu', name='dense_1')
        self.dense_2 = layers.Dense(64, activation='relu', name='dense_2')
        self.pred_layer = layers.Dense(10, name='predictions')

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        return self.pred_layer(x)


def get_model():
    return ThreeLayerMLP(name='3_layer_mlp')


model = get_model()

In [67]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=['acc'])

history = model.fit(x_train, y_train, batch_size=64, epochs=1)

Train on 60000 samples
60000/60000 [==============================] - 1s 23us/sample - loss: 0.3110 - acc: 0.9095


#### First

In [68]:
model.save_weights('submodel_tf', save_format='tf')

In [69]:
predictions = model.predict(x_test)
first_batch_loss = model.train_on_batch(x_train[:64], y_train[:64])[0]

In [70]:
new_model = get_model()
new_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=['acc'])

new_model.load_weights('submodel_tf')

new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

new_first_batch_loss = new_model.train_on_batch(x_train[:64], y_train[:64])
assert first_batch_loss == new_first_batch_loss[0]

#### Second

In [71]:
model.save('save_model_custom',save_format='tf')

new_model = keras.models.load_model('save_model_custom')

INFO:tensorflow:Assets written to: save_model_custom/assets


#### Third

In [72]:
tf.saved_model.save(model,'tf_save_model_custom')
restored_saved_model = keras.models.load_model('tf_save_model_custom')

INFO:tensorflow:Assets written to: tf_save_model_custom/assets
